In [ ]:
      !pip install tensorflow-federated

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Reshape
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.metrics import log_loss


In [ ]:
import pandas as pd
import numpy as np

# Chemins vers les datasets
drebin_data_path = '/content/drive/MyDrive/datafed1/drebin_processed.csv'
malgenome_data_path = '/content/drive/MyDrive/datafed1/malgenome2_processed.csv'
tuandromd_data_path = '/content/drive/MyDrive/datafed1/TUANDROMD_processed.csv'
kronodroid_data_path = '/content/drive/MyDrive/datafed1/kronodroid.csv'

# Chargement des données originales
Drebin_data = pd.read_csv(drebin_data_path)
Malgenome_data = pd.read_csv(malgenome_data_path)
TUANDROMD_data = pd.read_csv(tuandromd_data_path)
Kronodroid_data = pd.read_csv(kronodroid_data_path)

# la dernière colonne contient les étiquettes, séparation des caractéristiques et les étiquettes
X_drebin = Drebin_data.iloc[:, :-1]
y_drebin = Drebin_data.iloc[:, -1]
X_malgenome = Malgenome_data.iloc[:, :-1]
y_malgenome = Malgenome_data.iloc[:, -1]
X_tuandromd = TUANDROMD_data.iloc[:, :-1]
y_tuandromd = TUANDROMD_data.iloc[:, -1]
X_kronodroid = Kronodroid_data.iloc[:, :-1]
y_kronodroid = Kronodroid_data.iloc[:, -1]

# Liste complète des caractéristiques de tous les jeux de données
all_features = list(set(X_drebin.columns).union(set(X_malgenome.columns)).union(set(X_tuandromd.columns)).union(set(X_kronodroid.columns)))


# Réindexer les DataFrames pour inclure toutes les caractéristiques, en remplissant les valeurs manquantes par 0
def reindex_with_all_features(df, all_features):
    current_features = df.columns.tolist()
    missing_features = list(set(all_features) - set(current_features))
    df = df.reindex(columns=current_features + missing_features, fill_value=0)
    return df

X_drebin = reindex_with_all_features(X_drebin, all_features)
X_malgenome = reindex_with_all_features(X_malgenome, all_features)
X_tuandromd = reindex_with_all_features(X_tuandromd, all_features)
X_kronodroid = reindex_with_all_features(X_kronodroid, all_features)


# Ajouter les labels aux DataFrames alignés
X_drebin['label'] = y_drebin
X_malgenome['label'] = y_malgenome
X_tuandromd['label'] = y_tuandromd
X_kronodroid['label'] = y_kronodroid

# Sauvegarder chaque DataFrame avec les labels dans des fichiers CSV
X_drebin.to_csv('/content/drive/MyDrive/datafed1/Xy_drebin_processed.csv', index=False)
X_malgenome.to_csv('/content/drive/MyDrive/datafed1/Xy_malgenome_processed.csv', index=False)
X_tuandromd.to_csv('/content/drive/MyDrive/datafed1/Xy_tuandromd_processed.csv', index=False)
X_kronodroid.to_csv('/content/drive/MyDrive/datafed1/Xy_kronodroid_processed.csv', index=False)

# Concaténer les DataFrames alignés et les étiquettes
X_test = pd.concat([X_drebin.drop(columns=['label']), X_malgenome.drop(columns=['label']), X_tuandromd.drop(columns=['label']), X_kronodroid.drop(columns=['label'])])
y_test = pd.concat([y_drebin, y_malgenome, y_tuandromd, y_kronodroid])

# Convertir de nouveau en array numpy si nécessaire
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)
print(all_features)

Shape of X_test: (93655, 687)
Shape of y_test: (93655,)
['READ_CONTACTS', 'ANT_ADMIN', 'android.intent.action.SEND', 'DexClassLoader', 'rt_sigtimedwait', 'Landroid/telephony/TelephonyManager;->getNetworkOperator', 'SYS_344', 'CHANGE_DISPLAY_MODE', 'UNINSTALL_SHORTCUT', 'SYS_317', 'rt_sigreturn', 'BIND_CONDITION_PROVIDER_SERVICE', 'READ_PHONE_STATE', 'GOOGLE_AUTH', 'GET_TOP_ACTIVITY_INFO', 'Ljavax/crypto/Cipher;->doFinal', 'Ljava.lang.Class.cast', 'pselect6', 'READ_INPUT_STATE', 'BIND_CONTROLS', 'bindService', 'clone', 'MESSAGE', 'TelephonyManager.isNetworkRoaming', 'FACTORY_TEST', 'clock_settime', 'SET_PREFERRED_APPLICATIONS', 'fremovexattr', 'restart_syscall', 'android.intent.action.SEND_MULTIPLE', 'INSTALL_PACKAGES', 'swapoff', 'CALL_PHONE', 'WRITE_SECURE_SETTINGS', 'PERMISSION', 'epoll_ctl', 'dangerous', 'android.intent.action.REBOOT', 'delete_module', 'SEND_RESPOND_VIA_MESSAGE', 'Runtime.exec', 'android.content.pm.PackageInfo', 'Ljavax.crypto.spec.SecretKeySpec', 'nr_syscalls', 'AC

In [ ]:

# Chemins vers les datasets
drebin_data_path = '/content/drive/MyDrive/datafed1/Xy_drebin_processed.csv'
malgenome_data_path = '/content/drive/MyDrive/datafed1/Xy_malgenome_processed.csv'
tuandromd_data_path = '/content/drive/MyDrive/datafed1/Xy_tuandromd_processed.csv'
kronodroid_data_path = '/content/drive/MyDrive/datafed1/Xy_kronodroid_processed.csv'

# Chargement des données
Drebin_data = pd.read_csv(drebin_data_path)
Malgenome_data = pd.read_csv(malgenome_data_path)
TUANDROMD_data = pd.read_csv(tuandromd_data_path)
Kronodroid_data = pd.read_csv(kronodroid_data_path)



<ipython-input-3-ddca1be28006>:8: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  Drebin_data = pd.read_csv(drebin_data_path)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import f1_score, precision_score, log_loss

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    cnn_model = create_cnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré
for round_num in range(1, 51):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Afficher les métriques pour chaque dataset
    for i, client_data in enumerate(federated_train_data):
        metrics_client = metrics['client_work']['train']
        print(f'Dataset {dataset_names[i]}: Metrics: {metrics_client}')

# Préparer les données de test pour l'évaluation
X_test = np.concatenate((X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test)).reshape(-1, max_features, 1)
y_test = np.concatenate((y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test))

# Assurez-vous que 'create_cnn_model' est définie pour créer le modèle CNN
keras_cnn_model = create_cnn_model((max_features, 1))

# Assigner les poids du modèle fédéré au modèle Keras
global_model_weights = trainer.get_model_weights(state)
global_model_weights.assign_weights_to(keras_cnn_model)

# Compiler le modèle Keras avant l'évaluation
keras_cnn_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Évaluation sur les données de test
cnn_accuracy = keras_cnn_model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {cnn_accuracy[1]}')

# Prédictions
y_cnn_pred = keras_cnn_model.predict(X_test)
y_cnn_pred = (y_cnn_pred > 0.5).astype(int)

# Calcul du score F1 et de la précision
cnn_f1 = f1_score(y_test, y_cnn_pred, average='weighted')
cnn_precision = precision_score(y_test, y_cnn_pred, average='weighted')

# Calcul du loss
cnn_log_loss_value = log_loss(y_test, y_cnn_pred)

print(f'F1 Score: {cnn_f1}')
print(f'Precision: {cnn_precision}')
print(f'Log Loss: {cnn_log_loss_value}')


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.7384164), ('loss', 0.6066188), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Dataset Drebin: Metrics: OrderedDict([('binary_accuracy', 0.7384164), ('loss', 0.6066188), ('num_examples', 7489), ('num_batches', 376)])
Dataset Malgenome: Metrics: OrderedDict([('binary_accuracy', 0.7384164), ('loss', 0.6066188), ('num_examples', 7489), ('num_batches', 376)])
Dataset TUANDROMD: Metrics: OrderedDict([('binary_accuracy', 0.7384164), ('loss', 0.6066188), ('num_examples', 7489), ('num_batches', 376)])
Dataset Kronodroid: Metrics: OrderedDict([('binary_accuracy', 0.7384164), ('loss', 0.6066188), ('num_examples', 7489), ('num_batches', 376)])
--- Round 2 ---
Round 2, Metrics: OrderedDict([('distributor', ()), ('client_work', 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import f1_score, precision_score, log_loss

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données
# Drebin_data, Malgenome_data, TUANDROMD_data, Kronodroid_data doivent être définis en amont
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    cnn_model = create_cnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré avec évaluation séparée pour chaque dataset
for round_num in range(1, 51):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Évaluer le modèle global sur les données de test de chaque client séparément
    for i, (X_test, y_test) in enumerate(zip(
            [X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test],
            [y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test])):
        keras_cnn_model = create_cnn_model((max_features, 1))
        global_model_weights = trainer.get_model_weights(state)
        global_model_weights.assign_weights_to(keras_cnn_model)
        keras_cnn_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
        client_accuracy = keras_cnn_model.evaluate(X_test, y_test, verbose=0)
        print(f'Dataset {dataset_names[i]}: Accuracy: {client_accuracy[1]}')

# Préparer les données de test pour l'évaluation globale
X_test = np.concatenate((X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test)).reshape(-1, max_features, 1)
y_test = np.concatenate((y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test))

# Assurez-vous que 'create_cnn_model' est définie pour créer le modèle CNN
keras_cnn_model = create_cnn_model((max_features, 1))

# Assigner les poids du modèle fédéré au modèle Keras
global_model_weights.assign_weights_to(keras_cnn_model)

# Compiler le modèle Keras avant l'évaluation
keras_cnn_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Évaluer le modèle global sur les données de test globales
test_accuracy = keras_cnn_model.evaluate(X_test, y_test, verbose=0)
print(f'Global Test Accuracy: {test_accuracy[1]}')

# Prédire les étiquettes pour les données de test globales
y_pred = keras_cnn_model.predict(X_test).reshape(-1)
y_true = y_test.reshape(-1)

# Calculer la précision, le rappel, le score F1, et la perte logarithmique
precision = precision_score(y_true, y_pred.round())
recall = recall_score(y_true, y_pred.round())
f1 = f1_score(y_true, y_pred.round())
logloss = log_loss(y_true, y_pred)

print(f'Global Test Precision: {precision}')
print(f'Global Test Recall: {recall}')
print(f'Global Test F1 Score: {f1}')
print(f'Global Test Log Loss: {logloss}')


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.7560422), ('loss', 0.58408237), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Dataset Drebin: Accuracy: 0.31310343742370605
Dataset Malgenome: Accuracy: 0.6644737124443054
Dataset TUANDROMD: Accuracy: 0.2217245250940323
Dataset Kronodroid: Accuracy: 0.8804069757461548
--- Round 2 ---
Round 2, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.8792896), ('loss', 0.32414103), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Dataset Drebin: Accuracy: 0.6620689630508423
Dataset Malgenome: Accuracy: 0.6013157963752747
Datase

In [ ]:

from sklearn.metrics import recall_score
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import f1_score, precision_score, log_loss

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données
# Drebin_data, Malgenome_data, TUANDROMD_data, Kronodroid_data doivent être définis en amont
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    cnn_model = create_cnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()
# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré avec évaluation séparée pour chaque dataset
for round_num in range(1, 51):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Évaluer le modèle global sur les données de test de chaque client séparément
    for i, (X_test, y_test) in enumerate(zip(
            [X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test],
            [y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test])):
        keras_cnn_model = create_cnn_model((max_features, 1))
        global_model_weights = trainer.get_model_weights(state)
        global_model_weights.assign_weights_to(keras_cnn_model)
        keras_cnn_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

        # Évaluation du modèle
        test_loss, test_accuracy = keras_cnn_model.evaluate(X_test, y_test, verbose=0)

        # Prédiction
        y_pred = keras_cnn_model.predict(X_test).reshape(-1)
        y_true = y_test.reshape(-1)

        # Calcul des métriques
        precision = precision_score(y_true, y_pred.round())
        recall = recall_score(y_true, y_pred.round())
        f1 = f1_score(y_true, y_pred.round())
        logloss = log_loss(y_true, y_pred)

        # Affichage des résultats pour chaque dataset
        print(f'Dataset {dataset_names[i]}  Round {round_num}:')
        print(f'  Accuracy: {test_accuracy}')
        print(f'  Precision: {precision}')
        print(f'  Recall: {recall}')
        print(f'  F1 Score: {f1}')
        print(f'  Log Loss: {logloss}')


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.81613034), ('loss', 0.5561935), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
46/46 [==============================] - 2s 29ms/step
Dataset Drebin après Round 1:
  Accuracy: 0.2944827675819397
  Precision: 0.6005154639175257
  Recall: 0.21162579473206175
  F1 Score: 0.31296171927468097
  Log Loss: 0.759046084281309
24/24 [==============================] - 1s 20ms/step
Dataset Malgenome après Round 1:
  Accuracy: 0.4578947424888611
  Precision: 0.31295843520782396
  Recall: 0.4942084942084942
  F1 Score: 0.3832335329341317
  Log Loss: 0.7196292728868903
28/28 [==============================] - 0s 11ms/step
Dataset TUANDROMD après Round 1:
  Accuracy: 0.2351623773574829
  Precision: 0.6229508196721312
  Recall: 0.15

In [ ]:
# Préparer les données de test pour l'évaluation globale
X_test = np.concatenate((X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test)).reshape(-1, max_features, 1)
y_test = np.concatenate((y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test))

# Assurez-vous que 'create_cnn_model' est définie pour créer le modèle CNN
keras_cnn_model = create_cnn_model((max_features, 1))

# Assigner les poids du modèle fédéré au modèle Keras
global_model_weights.assign_weights_to(keras_cnn_model)

# Compiler le modèle Keras avant l'évaluation
keras_cnn_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Évaluer le modèle global sur les données de test globales
test_accuracy = keras_cnn_model.evaluate(X_test, y_test, verbose=0)
print(f'Global Test Accuracy: {test_accuracy[1]}')

# Prédire les étiquettes pour les données de test globales
y_pred = keras_cnn_model.predict(X_test).reshape(-1)
y_true = y_test.reshape(-1)

# Calculer la précision, le rappel, le score F1, et la perte logarithmique
precision = precision_score(y_true, y_pred.round())
recall = recall_score(y_true, y_pred.round())
f1 = f1_score(y_true, y_pred.round())
logloss = log_loss(y_true, y_pred)

print(f'Global Test Precision: {precision}')
print(f'Global Test Recall: {recall}')
print(f'Global Test F1 Score: {f1}')
print(f'Global Test Log Loss: {logloss}')


Global Test Accuracy: 0.9766163229942322
586/586 [==============================] - 9s 14ms/step
Global Test Precision: 0.9714204329661983
Global Test Recall: 0.9867862654320988
Global Test F1 Score: 0.979043062200957
Global Test Log Loss: 0.107174487498094


In [ ]:
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import f1_score, precision_score, log_loss

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    cnn_model = create_cnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré avec évaluation séparée pour chaque modèle local
for round_num in range(1, 51):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Évaluer chaque modèle local (avant agrégation) sur les données de test de tous les clients
    for client_id, (X_train, y_train) in enumerate(zip(
            [X_drebin_train, X_malgenome_train, X_tuandromd_train, X_kronodroid_train],
            [y_drebin_train, y_malgenome_train, y_tuandromd_train, y_kronodroid_train])):
        # Créez et compilez un modèle pour chaque client
        local_model = create_cnn_model((max_features, 1))
        local_weights = trainer.get_model_weights(state)  # Récupérez les poids du modèle local pour le client
        local_weights.assign_weights_to(local_model)
        local_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

        # Évaluer le modèle local sur toutes les données de test
        for i, (X_test, y_test) in enumerate(zip(
                [X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test],
                [y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test])):

            test_loss, test_accuracy = local_model.evaluate(X_test, y_test, verbose=0)

            # Prédiction
            y_pred = local_model.predict(X_test).reshape(-1)
            y_true = y_test.reshape(-1)

            # Calcul des métriques
            precision = precision_score(y_true, y_pred.round())
            recall = recall_score(y_true, y_pred.round())
            f1 = f1_score(y_true, y_pred.round())

            print(f"Client {dataset_names[client_id]}, Test Dataset {dataset_names[i]}: Loss: {test_loss}, Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.7392175), ('loss', 0.5678077), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
46/46 [==============================] - 1s 12ms/step
Client Drebin, Test Dataset Drebin: Loss: 0.7408687472343445, Accuracy: 0.3234482705593109, Precision: 0.6829268292682927, Recall: 0.20345140781108084, F1: 0.31350594821553535
24/24 [==============================] - 0s 13ms/step
Client Drebin, Test Dataset Malgenome: Loss: 0.6847755908966064, Accuracy: 0.5789473652839661, Precision: 0.3151515151515151, Recall: 0.20077220077220076, F1: 0.24528301886792453
28/28 [==============================] - 0s 12ms/step
Client Drebin, Test Dataset TUANDROMD: Loss: 0.8033127188682556, Accuracy: 0.18812990188598633, Precision: 0.5882352941176471, Re

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import precision_score, recall_score, f1_score

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

# Préparation des données
def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données (les datasets doivent être fournis ici)
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Combiner les ensembles de test pour un test global commun
X_global_test = np.concatenate([X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test], axis=0)
y_global_test = np.concatenate([y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test], axis=0)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle Keras
def create_keras_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    cnn_model = create_keras_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Fonction pour évaluer le modèle local
def evaluate_local_model(state, input_shape, X_test, y_test):
    # Créer un nouveau modèle pour chaque client
    local_model = create_cnn_model(input_shape)

    # Extraire les poids du modèle global du client
    local_weights = trainer.get_model_weights(state)
    local_weights.assign_weights_to(local_model)

    # Compiler le modèle avec un nouvel optimiseur et les mêmes paramètres de perte
    local_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

    # Évaluer le modèle sur les données de test globales
    test_loss, test_accuracy = local_model.evaluate(X_test, y_test, verbose=0)

    # Prédiction pour calculer d'autres métriques
    y_pred = local_model.predict(X_test).reshape(-1)
    y_true = y_test.reshape(-1)

    # Calcul des métriques
    precision = precision_score(y_true, y_pred.round())
    recall = recall_score(y_true, y_pred.round())
    f1 = f1_score(y_true, y_pred.round())

    return test_loss, test_accuracy, precision, recall, f1

# Entraînement fédéré
for round_num in range(1, 51):  # Par exemple, 50 rounds
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Évaluation locale pour chaque client
    for client_id, client_data in enumerate(federated_train_data):
        # Créer et évaluer un modèle séparé pour chaque client
        test_loss, test_accuracy, precision, recall, f1 = evaluate_local_model(
            state, (max_features, 1), X_global_test, y_global_test)

        # Afficher les résultats pour chaque client
        print(f"Client {dataset_names[client_id]}, Loss: {test_loss}, Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.81666446), ('loss', 0.5081826), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
586/586 [==============================] - 11s 18ms/step
Client Drebin, Loss: 0.38875046372413635, Accuracy: 0.844162106513977, Precision: 0.8693841118714668, Recall: 0.8454861111111112, F1: 0.8572685932228252
586/586 [==============================] - 10s 17ms/step
Client Malgenome, Loss: 0.38875046372413635, Accuracy: 0.844162106513977, Precision: 0.8693841118714668, Recall: 0.8454861111111112, F1: 0.8572685932228252
586/586 [==============================] - 10s 17ms/step
Client TUANDROMD, Loss: 0.38875046372413635, Accuracy: 0.844162106513977, Precision: 0.8693841118714668, Recall: 0.8454861111111112, F1: 0.8572685932228252
586/586 [

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

# Fonction pour préparer les données
def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Combiner les ensembles de test pour un test global commun
X_global_test = np.concatenate([X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test], axis=0)
y_global_test = np.concatenate([y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test], axis=0)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Fonction pour créer le modèle CNN
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

# Préparer les données pour chaque client
def prepare_federated_data():
    return [
        dataset_to_tff_format(X_drebin_train, y_drebin_train),
        dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
        dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
        dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
    ]

# Initialisation du modèle fédéré TFF
def model_fn():
    cnn_model = create_cnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définir la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()

# Entraînement fédéré
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Préparer les données
federated_train_data = prepare_federated_data()

# Fonction pour évaluer le modèle local
def evaluate_local_model(state, input_shape, X_test, y_test):
    # Créer un nouveau modèle pour chaque client
    local_model = create_cnn_model(input_shape)

    # Extraire les poids du modèle global
    model_weights = trainer.get_model_weights(state)

    # Assigner les poids au modèle local
    try:
        local_model.set_weights(model_weights)
    except ValueError as e:
        print(f"Error in setting weights: {e}")
        return None, None, None, None, None

    # Compiler le modèle local
    local_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

    # Évaluer le modèle local sur les données de test globales
    test_loss, test_accuracy = local_model.evaluate(X_test, y_test, verbose=0)

    # Prédiction pour calculer d'autres métriques
    y_pred = local_model.predict(X_test).reshape(-1)
    y_true = y_test.reshape(-1)

    # Calcul des métriques
    precision = precision_score(y_true, y_pred.round())
    recall = recall_score(y_true, y_pred.round())
    f1 = f1_score(y_true, y_pred.round())

    return test_loss, test_accuracy, precision, recall, f1

# Entraînement et évaluation
for round_num in range(1, 51):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Évaluation locale pour chaque client
    for client_id, client_data in enumerate(federated_train_data):
        test_loss, test_accuracy, precision, recall, f1 = evaluate_local_model(
            state, (max_features, 1), X_global_test, y_global_test)

        # Afficher les résultats pour chaque client
        if test_loss is not None:
            print(f"Client {dataset_names[client_id]}, Loss: {test_loss}, Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")
        else:
            print(f"Client {dataset_names[client_id]} encountered an error during evaluation.")


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.82480973), ('loss', 0.5236198), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Error in setting weights: You called `set_weights(weights)` on layer "sequential_1" with a weight list of length 2, but the layer was expecting 12 weights. Provided weights: ModelWeights(trainable=[array([[[-0.04500099, -0.0...
Client Drebin encountered an error during evaluation.
Error in setting weights: You called `set_weights(weights)` on layer "sequential_2" with a weight list of length 2, but the layer was expecting 12 weights. Provided weights: ModelWeights(trainable=[array([[[-0.04500099, -0.0...
Client Malgenome encountered an error during evaluation.
Error in setting weights: You called `set_weights(weights)` on layer "sequenti

In [ ]:
######### Le bon
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import f1_score, precision_score, log_loss

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Combiner les ensembles de test pour un test global commun
X_global_test = np.concatenate([X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test], axis=0)
y_global_test = np.concatenate([y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test], axis=0)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    cnn_model = create_cnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré avec évaluation séparée pour chaque modèle local
for round_num in range(1, 50):  # 50 tours de communication
    # Effectuer un tour d'entraînement

    state, metrics = trainer.next(state, federated_train_data)
    print(f'--- Round {round_num} ---')
    print(f'Round {round_num}, Metrics: {metrics}')

    # Extraire les poids du modèle fédéré après l'entraînement
    model_weights = trainer.get_model_weights(state)

    # Évaluer chaque modèle localement
    for client_id, (X_train, y_train, X_test, y_test) in enumerate(zip(
            [X_drebin_train, X_malgenome_train, X_tuandromd_train, X_kronodroid_train],
            [y_drebin_train, y_malgenome_train, y_tuandromd_train, y_kronodroid_train],
            [X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test],
            [y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test])):

        # Créez et compilez un modèle pour chaque client
        local_model = create_cnn_model((max_features, 1))

        # Charger les poids mis à jour du modèle fédéré
        local_model.set_weights(model_weights.trainable)

        # Compiler le modèle local
        local_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

        # Évaluer le modèle local sur les données de test du client
        test_loss, test_accuracy = local_model.evaluate(X_test, y_test, verbose=0)

        # Prédiction
        y_pred = local_model.predict(X_test).reshape(-1)
        y_true = y_test.reshape(-1)

        # Calcul des métriques
        precision = precision_score(y_true, y_pred.round())
        recall = recall_score(y_true, y_pred.round())
        f1 = f1_score(y_true, y_pred.round())

        print(f"Client {dataset_names[client_id]}, Loss: {test_loss}, Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

    # Évaluer le modèle global sur les données de test globales
    global_model = create_cnn_model((max_features, 1))
    global_model.set_weights(model_weights.trainable)
    global_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

    # Évaluation sur les données globales
    global_loss, global_accuracy = global_model.evaluate(X_global_test, y_global_test, verbose=0)

    # Prédiction globale
    y_global_pred = global_model.predict(X_global_test).reshape(-1)
    y_global_true = y_global_test.reshape(-1)

    # Calcul des métriques globales
    global_precision = precision_score(y_global_true, y_global_pred.round())
    global_recall = recall_score(y_global_true, y_global_pred.round())
    global_f1 = f1_score(y_global_true, y_global_pred.round())

    print(f"Global Model, Loss: {global_loss}, Accuracy: {global_accuracy}, Precision: {global_precision}, Recall: {global_recall}, F1: {global_f1}")


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.8305515), ('loss', 0.5292536), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
46/46 [==============================] - 1s 11ms/step
Client Drebin, Loss: 0.5873587727546692, Accuracy: 0.7558620572090149, Precision: 0.7610062893081762, Recall: 0.989100817438692, F1: 0.8601895734597157
24/24 [==============================] - 0s 12ms/step
Client Malgenome, Loss: 0.922274649143219, Accuracy: 0.3618420958518982, Precision: 0.3481182795698925, Recall: 1.0, F1: 0.5164506480558325
28/28 [==============================] - 0s 12ms/step
Client TUANDROMD, Loss: 0.568077564239502, Accuracy: 0.8163493871688843, Precision: 0.8168539325842696, Recall: 0.9986263736263736, F1: 0.8986402966625463
489/489 [============================

In [ ]:
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.metrics import f1_score, precision_score, log_loss

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalercnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le Conv1D
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Combiner les ensembles de test pour un test global commun
X_global_test = np.concatenate([X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test], axis=0)
y_global_test = np.concatenate([y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test], axis=0)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    cnn_model = create_cnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        cnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.SGD()

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré avec évaluation séparée pour chaque modèle local
for round_num in range(1, 51):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

  # Évaluer chaque modèle local (avant agrégation) sur l'ensemble de test global
for client_id, (X_train, y_train) in enumerate(zip(
        [X_drebin_train, X_malgenome_train, X_tuandromd_train, X_kronodroid_train],
        [y_drebin_train, y_malgenome_train, y_tuandromd_train, y_kronodroid_train])):

    # Créez et compilez un modèle pour chaque client
    local_model = create_cnn_model((max_features, 1))

    # Charger les poids avant agrégation pour chaque client spécifique
    local_weights = trainer.get_model_weights(state).trainable
    local_model.set_weights(local_weights)

    # Compiler le modèle local
    local_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

    # Évaluer le modèle local sur les données de test globales
    test_loss, test_accuracy = local_model.evaluate(X_global_test, y_global_test, verbose=0)

    # Prédiction
    y_pred = local_model.predict(X_global_test).reshape(-1)
    y_true = y_global_test.reshape(-1)

    # Calcul des métriques
    precision = precision_score(y_true, y_pred.round())
    recall = recall_score(y_true, y_pred.round())
    f1 = f1_score(y_true, y_pred.round())

    print(f"Client {dataset_names[client_id]}, Loss: {test_loss}, Accuracy: {test_accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")


--- Round 1 ---
Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.83081853), ('loss', 0.52314514), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
--- Round 2 ---
Round 2, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.88903725), ('loss', 0.29468063), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
--- Round 3 ---
Round 3, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.9216184), ('loss', 0.21493086), ('num_examples', 7489), ('num_batches', 376)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight'

In [ ]:

# Préparer les données de test pour l'évaluation
X_test = np.concatenate((X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test)).reshape(-1, max_features, 1)
y_test = np.concatenate((y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test))

# Assurez-vous que 'create_cnn_model' est définie pour créer le modèle CNN
keras_cnn_model = create_cnn_model((max_features, 1))

# Assigner les poids du modèle fédéré au modèle Keras
global_model_weights = trainer.get_model_weights(state)
global_model_weights.assign_weights_to(keras_cnn_model)

# Compiler le modèle Keras avant l'évaluation
keras_cnn_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Évaluation sur les données de test
cnn_accuracy = keras_cnn_model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {cnn_accuracy[1]}')

# Prédictions
y_cnn_pred = keras_cnn_model.predict(X_test)
y_cnn_pred = (y_cnn_pred > 0.5).astype(int)

# Calcul du score F1 et de la précision
cnn_f1 = f1_score(y_test, y_cnn_pred, average='weighted')
cnn_precision = precision_score(y_test, y_cnn_pred, average='weighted')

# Calcul du loss
cnn_log_loss_value = log_loss(y_test, y_cnn_pred)

print(f'F1 Score: {cnn_f1}')
print(f'Precision_SGD_cnn: {cnn_precision}')
print(f'Log Loss: {cnn_log_loss_value}')


Accuracy: 0.9750680923461914
586/586 [==============================] - 10s 16ms/step
F1 Score: 0.9750620359345173
Precision_SGD_cnn: 0.9750654579673799
Log Loss: 0.8986378801301433


In [ ]:
keras_cnn_model.save('/content/drive/MyDrive/trained_models/mycnn19750__model.keras')


In [ ]:
import tensorflow as tf

# Charger le modèle Keras
model = tf.keras.models.load_model('/content/drive/MyDrive/trained_models/mycnn19781__model.keras')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Sauvegarder le modèle TFLite
with open('/content/drive/MyDrive/trained_models/mycnn19781_model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
# Installation de apktool sur Google Colab
!wget https://raw.githubusercontent.com/iBotPeaches/Apktool/master/scripts/linux/apktool -O /usr/local/bin/apktool
!wget https://bitbucket.org/iBotPeaches/apktool/downloads/apktool_2.6.0.jar -O /usr/local/bin/apktool.jar
!chmod +x /usr/local/bin/apktool
!chmod +x /usr/local/bin/apktool.ja

--2024-08-12 11:42:07--  https://raw.githubusercontent.com/iBotPeaches/Apktool/master/scripts/linux/apktool
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2412 (2.4K) [text/plain]
Saving to: ‘/usr/local/bin/apktool’

/usr/local/bin/apkt 100%[===================>]   2.36K  --.-KB/s    in 0s      

2024-08-12 11:42:07 (26.9 MB/s) - ‘/usr/local/bin/apktool’ saved [2412/2412]

--2024-08-12 11:42:07--  https://bitbucket.org/iBotPeaches/apktool/downloads/apktool_2.6.0.jar
Resolving bitbucket.org (bitbucket.org)... 104.192.142.24, 104.192.142.26, 104.192.142.25, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.142.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/0becf6a1-1706-4f2e-9ae6-

In [ ]:
import os
import subprocess
import xml.etree.ElementTree as ET

def analyze_apk(apk_path):
    output_folder = apk_path + '_output'

    # Décompiler l'APK
    subprocess.run(['apktool', 'd', '-f', apk_path, '-o', output_folder])

    manifest_file = os.path.join(output_folder, 'AndroidManifest.xml')

    if not os.path.exists(manifest_file):
        print(f'Le fichier {manifest_file} n\'a pas été trouvé.')
        return

    # Analyser le fichier AndroidManifest.xml
    tree = ET.parse(manifest_file)
    root = tree.getroot()

    # Extraire les permissions
    permissions = set()
    for elem in root.iter('uses-permission'):
        permission = elem.get('{http://schemas.android.com/apk/res/android}name')
        if permission:
            permissions.add(permission.split('.')[-1])  # Extraire le nom de la permission

    # Extraire les permissions utilisées
    used_permissions = set()
    for elem in root.iter('permission'):
        permission = elem.get('{http://schemas.android.com/apk/res/android}name')
        if permission:
            used_permissions.add(permission.split('.')[-1])

    # Extraire les composants de l'application
    app_components = set()
    for component in ['activity', 'service', 'receiver', 'provider']:
        for elem in root.iter(component):
            component_name = elem.get('{http://schemas.android.com/apk/res/android}name')
            if component_name:
                app_components.add(component_name.split('.')[-1])

    # Extraire les composants matériels
    hardware_components = set()
    for elem in root.iter('uses-feature'):
        feature = elem.get('{http://schemas.android.com/apk/res/android}name')
        if feature:
            hardware_components.add(feature.split('.')[-1])

    # Extraire les intents filtrés
    filtered_intents = set()
    for elem in root.iter('intent-filter'):
        for action in elem.iter('action'):
            intent = action.get('{http://schemas.android.com/apk/res/android}name')
            if intent:
                filtered_intents.add(intent.split('.')[-1])

    # Extraire les appels API et les API systèmes
    smali_folder = os.path.join(output_folder, 'smali')
    api_calls = set()
    system_apis = set()

    for subdir, _, files in os.walk(smali_folder):
        for file in files:
            if file.endswith('.smali'):
                file_path = os.path.join(subdir, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        if 'invoke-' in line:  # Trouver les appels API
                            api_calls.add(line.strip())
                        if 'Landroid/os/' in line or 'Ljava/io/' in line:  # Trouver les API systèmes
                            system_apis.add(line.strip())

    # Combiner toutes les informations dans une variable sans spécifier la source
    combined_info = list(permissions.union(used_permissions, app_components, hardware_components, filtered_intents, api_calls, system_apis))

    # Afficher les informations extraites
    print(f'Permissions utilisées : {used_permissions}')
    print(f'Composants de l\'application : {app_components}')
    print(f'Composants matériels : {hardware_components}')
    print(f'Intents filtrés : {filtered_intents}')
    print(f'Permissions demandées : {permissions}')
    print(f'Appels API trouvés : {api_calls}')
    print(f'API systèmes trouvés : {system_apis}')


    return combined_info
    print(f'Informations combinées : {combined_info}')

    # Nettoyer les fichiers temporaires
    subprocess.run(['rm', '-rf', output_folder])


In [ ]:
import os
import subprocess
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf

# Fonction pour extraire les caractéristiques d'un fichier APK
def analyze_apk(apk_path):
    output_folder = apk_path + '_output'

    # Décompiler l'APK
    subprocess.run(['apktool', 'd', '-f', apk_path, '-o', output_folder])

    manifest_file = os.path.join(output_folder, 'AndroidManifest.xml')

    if not os.path.exists(manifest_file):
        print(f'Le fichier {manifest_file} n\'a pas été trouvé.')
        return []

    # Analyser le fichier AndroidManifest.xml
    tree = ET.parse(manifest_file)
    root = tree.getroot()

    # Extraire les permissions
    permissions = set()
    for elem in root.iter('uses-permission'):
        permission = elem.get('{http://schemas.android.com/apk/res/android}name')
        if permission:
            permissions.add(permission.split('.')[-1])  # Extraire le nom de la permission

    # Extraire les composants de l'application
    app_components = set()
    for component in ['activity', 'service', 'receiver', 'provider']:
        for elem in root.iter(component):
            component_name = elem.get('{http://schemas.android.com/apk/res/android}name')
            if component_name:
                app_components.add(component_name.split('.')[-1])

    # Extraire les composants matériels
    hardware_components = set()
    for elem in root.iter('uses-feature'):
        feature = elem.get('{http://schemas.android.com/apk/res/android}name')
        if feature:
            hardware_components.add(feature.split('.')[-1])

    # Extraire les intents filtrés
    filtered_intents = set()
    for elem in root.iter('intent-filter'):
        for action in elem.iter('action'):
            intent = action.get('{http://schemas.android.com/apk/res/android}name')
            if intent:
                filtered_intents.add(intent.split('.')[-1])

    # Extraire les appels API et les API systèmes
    smali_folder = os.path.join(output_folder, 'smali')
    api_calls = set()
    system_apis = set()

    for subdir, _, files in os.walk(smali_folder):
        for file in files:
            if file.endswith('.smali'):
                file_path = os.path.join(subdir, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        if 'invoke-' in line:  # Trouver les appels API
                            api_calls.add(line.strip())
                        if 'Landroid/os/' in line or 'Ljava/io/' in line:  # Trouver les API systèmes
                            system_apis.add(line.strip())

    # Combiner toutes les informations dans une variable sans spécifier la source
    combined_info = list(permissions.union(app_components, hardware_components, filtered_intents, api_calls, system_apis))

    # Nettoyer les fichiers temporaires
    subprocess.run(['rm', '-rf', output_folder])

    return combined_info

# Chemins vers le modèle et les paramètres du scaler
model_path = '/content/drive/MyDrive/trained_models/mycnn19781__model.keras'
scaler_mean_path = '/content/drive/MyDrive/datafed1/scalercnn1.npy_mean.npy'
scaler_scale_path = '/content/drive/MyDrive/datafed1/scalercnn1.npy_scale.npy'

# Charger le modèle entraîné
model = tf.keras.models.load_model(model_path)


# Fonction pour préparer les données pour le modèle
def prepare_data(extracted_info, all_features):
    feature_vector = np.zeros(len(all_features))
    for i, feature in enumerate(all_features):
        if feature in extracted_info:
            feature_vector[i] = 1
    return feature_vector

# Chemin vers le fichier APK
apk_path = '/content/drive/MyDrive/datafed1/Lite_416.0.0.10.102.apk'
# Extraire les caractéristiques de l'APK
extracted_info = analyze_apk(apk_path)

# Préparer les données pour le modèle
feature_vector = prepare_data(extracted_info, all_features)

# Charger le scaler et normaliser les données
scaler_mean = np.load(scaler_mean_path)
scaler_scale = np.load(scaler_scale_path)
scaled_features = (feature_vector - scaler_mean) / scaler_scale

# Reshaper les données pour Conv1D
scaled_features = scaled_features.reshape(-1, len(all_features), 1).astype(np.float32)

# Faire la prédiction
prediction = model.predict(scaled_features)

# Afficher le résultat
print("Prediction:", prediction)

# Interpréter la prédiction
if prediction[0] > 0.5:
    print("Le APK est malveillant.")
else:
    print("Le APK est bénin.")


1/1 [==============================] - 0s 110ms/step
Prediction: [[5.357173e-11]]
Le APK est bénin.


In [ ]:
#RNN
####################################RNN#######################
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Reshape

# Préparation des données
scaler_save_path = '/content/drive/MyDrive/datafed1/scalerrnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

def make_federated_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    federated_train_data = [
        (X_train, y_train),
        (X_train, y_train),
        (X_train, y_train),
        (X_train, y_train)
    ]

    return federated_train_data, (X_test, y_test)

# Préparation des données pour chaque client (dataset)
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le LSTM
X_drebin_train = X_drebin_train.reshape(-1, max_features, 1).astype(np.float32)
X_drebin_test = X_drebin_test.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_train = X_malgenome_train.reshape(-1, max_features, 1).astype(np.float32)
X_malgenome_test = X_malgenome_test.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_train = X_tuandromd_train.reshape(-1, max_features, 1).astype(np.float32)
X_tuandromd_test = X_tuandromd_test.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_train = X_kronodroid_train.reshape(-1, max_features, 1).astype(np.float32)
X_kronodroid_test = X_kronodroid_test.reshape(-1, max_features, 1).astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Création du modèle Keras LSTM
def create_rnn_model(input_shape):
    model = Sequential([
        LSTM(32, return_sequences=True, input_shape=input_shape),
        LSTM(64),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    rnn_model = create_rnn_model((max_features, 1))
    return tff.learning.models.from_keras_model(
        rnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features, 1], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
    return tf.keras.optimizers.Adam()

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré
for round_num in range(1, 21):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Afficher les métriques pour chaque dataset
    for i, client_data in enumerate(federated_train_data):
        metrics_client = metrics['client_work']['train']
        print(f'Dataset {dataset_names[i]}: Metrics: {metrics_client}')


In [ ]:
# Préparer les données de test pour l'évaluation
X_test = np.concatenate((X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test)).reshape(-1, max_features, 1)
y_test = np.concatenate((y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test))

# Évaluation du modèle RNN global
keras_rnn_model = create_rnn_model((max_features, 1))

# Assigner les poids du modèle global au modèle Keras
global_model_weights = trainer.get_model_weights(state)
global_model_weights.assign_weights_to(keras_rnn_model)

# Compiler le modèle Keras avant l'évaluation
keras_rnn_model.compile(optimizer='adam', metrics=['accuracy'])

# Évaluation sur les données de test
rnn_accuracy = keras_rnn_model.evaluate(X_test, y_test)
print(f'Accuracy: {rnn_accuracy[1]}')

# Prédictions
y_rnn_pred = keras_rnn_model.predict(X_test)
y_rnn_pred = (y_rnn_pred > 0.5).astype(int)

# Calcul du score F1 et de la précision
rnn_f1 = f1_score(y_test, y_rnn_pred, average='weighted')
rnn_precision = precision_score(y_test, y_rnn_pred, average='weighted', zero_division=1)

# Calcul du loss
y_test_rnn_proba = keras_rnn_model.predict(X_test)  # Probabilités prédites par le modèle
rnn_log_loss_value = log_loss(y_test, y_test_rnn_proba)

print(f'F1 Score: {rnn_f1}')
print(f'Precision: {rnn_precision}')
print(f'Loss: {rnn_log_loss_value}')

In [ ]:
# DNN

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import f1_score, precision_score, log_loss

# Chemin pour sauvegarder le scaler
scaler_save_path = '/content/drive/MyDrive/datafed1/scalerdnn.npy'

def prepare_data(data):
    data = data.replace('?', np.nan)
    data = data.dropna()
    features = np.array(data.iloc[:, :-1])
    labels = data.iloc[:, -1]

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    # Sauvegarde des paramètres du scaler
    np.save(scaler_save_path + '_mean.npy', scaler.mean_)
    np.save(scaler_save_path + '_scale.npy', scaler.scale_)

    lb = LabelBinarizer()
    y = lb.fit_transform(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    return X_train, X_test, y_train, y_test

# Charger les données (assurez-vous que les variables Drebin_data, Malgenome_data, TUANDROMD_data, Kronodroid_data sont définies)
X_drebin_train, X_drebin_test, y_drebin_train, y_drebin_test = prepare_data(Drebin_data)
X_malgenome_train, X_malgenome_test, y_malgenome_train, y_malgenome_test = prepare_data(Malgenome_data)
X_tuandromd_train, X_tuandromd_test, y_tuandromd_train, y_tuandromd_test = prepare_data(TUANDROMD_data)
X_kronodroid_train, X_kronodroid_test, y_kronodroid_train, y_kronodroid_test = prepare_data(Kronodroid_data)

# Déterminer le nombre maximal de caractéristiques parmi les datasets
max_features = max(X_drebin_train.shape[1], X_malgenome_train.shape[1], X_tuandromd_train.shape[1], X_kronodroid_train.shape[1])

# Adapter les données pour le DNN
X_drebin_train = X_drebin_train.astype(np.float32)
X_drebin_test = X_drebin_test.astype(np.float32)
X_malgenome_train = X_malgenome_train.astype(np.float32)
X_malgenome_test = X_malgenome_test.astype(np.float32)
X_tuandromd_train = X_tuandromd_train.astype(np.float32)
X_tuandromd_test = X_tuandromd_test.astype(np.float32)
X_kronodroid_train = X_kronodroid_train.astype(np.float32)
X_kronodroid_test = X_kronodroid_test.astype(np.float32)

# Convertir les étiquettes en int64
y_drebin_train = y_drebin_train.astype(np.int64)
y_drebin_test = y_drebin_test.astype(np.int64)
y_malgenome_train = y_malgenome_train.astype(np.int64)
y_malgenome_test = y_malgenome_test.astype(np.int64)
y_tuandromd_train = y_tuandromd_train.astype(np.int64)
y_tuandromd_test = y_tuandromd_test.astype(np.int64)
y_kronodroid_train = y_kronodroid_train.astype(np.int64)
y_kronodroid_test = y_kronodroid_test.astype(np.int64)

# Fonction pour échantillonner les données
def sample_data(X, y, fraction=0.1):
    sample_size = int(len(X) * fraction)
    indices = np.random.choice(len(X), sample_size, replace=False)
    return X[indices], y[indices]

# Échantillonner les données pour réduire leur taille
X_drebin_train, y_drebin_train = sample_data(X_drebin_train, y_drebin_train)
X_malgenome_train, y_malgenome_train = sample_data(X_malgenome_train, y_malgenome_train)
X_tuandromd_train, y_tuandromd_train = sample_data(X_tuandromd_train, y_tuandromd_train)
X_kronodroid_train, y_kronodroid_train = sample_data(X_kronodroid_train, y_kronodroid_train)

# Définir les noms des datasets
dataset_names = ['Drebin', 'Malgenome', 'TUANDROMD', 'Kronodroid']

# Définir le modèle DNN
def create_dnn_model(input_shape):
    model = Sequential([
        Dense(units=128, activation='relu', input_shape=input_shape),
        Dense(units=64, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    return model

# Préparation des données fédérées pour TFF
def dataset_to_tff_format(X, y, batch_size=20):
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

federated_train_data = [
    dataset_to_tff_format(X_drebin_train, y_drebin_train),
    dataset_to_tff_format(X_malgenome_train, y_malgenome_train),
    dataset_to_tff_format(X_tuandromd_train, y_tuandromd_train),
    dataset_to_tff_format(X_kronodroid_train, y_kronodroid_train)
]

# Définition du modèle fédéré TFF
def model_fn():
    dnn_model = create_dnn_model((max_features,))
    return tff.learning.models.from_keras_model(
        dnn_model,
        input_spec=(tf.TensorSpec(shape=[None, max_features], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Définition de la fonction pour l'optimiseur des clients
def client_optimizer_fn():
   # return tf.keras.optimizers.Adam()
    return tf.keras.optimizers.SGD(learning_rate=0.01)

# Initialisation de l'itération fédérée
trainer = tff.learning.algorithms.build_weighted_fed_avg(model_fn, client_optimizer_fn)
state = trainer.initialize()

# Entraînement fédéré
for round_num in range(1, 51):  # Par exemple, 50 tours de communication
    print(f'--- Round {round_num} ---')
    state, metrics = trainer.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

    # Afficher les métriques pour chaque dataset
    for i, client_data in enumerate(federated_train_data):
        metrics_client = metrics['client_work']['train']
        print(f'Dataset {dataset_names[i]}: Metrics: {metrics_client}')

# Préparer les données de test pour l'évaluation
X_test = np.concatenate((X_drebin_test, X_malgenome_test, X_tuandromd_test, X_kronodroid_test)).astype(np.float32)
y_test = np.concatenate((y_drebin_test, y_malgenome_test, y_tuandromd_test, y_kronodroid_test))

# Assurez-vous que 'create_dnn_model' est définie pour créer le modèle DNN
keras_dnn_model = create_dnn_model((max_features,))

# Assigner les poids du modèle fédéré au modèle Keras
global_model_weights = trainer.get_model_weights(state)
global_model_weights.assign_weights_to(keras_dnn_model)

# Compiler le modèle Keras avant l'évaluation
#keras_dnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
keras_dnn_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
# Évaluation sur les données de test
dnn_accuracy = keras_dnn_model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {dnn_accuracy[1]}')

# Prédictions
y_dnn_pred = keras_dnn_model.predict(X_test)
y_dnn_pred = (y_dnn_pred > 0.5).astype(int)

# Calcul du score F1 et de la précision
dnn_f1 = f1_score(y_test, y_dnn_pred, average='weighted')
dnn_precision = precision_score(y_test, y_dnn_pred, average='weighted')

# Calcul du loss
dnn_log_loss_value = log_loss(y_test, y_dnn_pred)

print(f'F1 Score: {dnn_f1}')
print(f'Precision_dnn_sgd: {dnn_precision}')
print(f'Log Loss: {dnn_log_loss_value}')


In [ ]:
keras_dnn_model.save('/content/drive/MyDrive/trained_models/mydnn_sgd9740__model.keras')
